In [ ]:
import numpy as np
import time

In [ ]:
oxic_decay_base = 0.035
anoxic_decay_base = 0.00001
base_temp = 6.29
Q10_oxic = 3
Q10_anoxic = 3
density = 0.1
porosity = 0.3
k_param_a = 1586.8724
k_param_b = 8
t_extent = 3
annual_tsteps = 525600
output_interval = 1
x_extent = 3
y_extent = 27
spatial_step = 200
pond_depth = 0.25
mean_temp = 11.35
t_step_multi = 0.5
max_tstep = 5
z_extent = t_extent+2

col_wt_depth = np.zeros([x_extent, y_extent])
col_wt_depth_av = np.zeros([x_extent, y_extent])
col_wt_depth_av_su = np.zeros([x_extent, y_extent])
col_wt_sum = np.zeros([x_extent, y_extent])
col_wt_sum_su = np.zeros([x_extent, y_extent])
col_mass_per_area = np.zeros([x_extent, y_extent])
base_altitude = np.zeros([x_extent, y_extent])
water_table = np.zeros([x_extent, y_extent])
activation_status = np.empty([x_extent, y_extent],dtype=object)
water_change = np.ones([x_extent, y_extent])
wk_mean = np.ones([x_extent, y_extent])
wet_proportion = np.zeros([x_extent, y_extent, z_extent])
layer_mass = np.zeros([x_extent, y_extent, z_extent, 3])
layer_attributes = np.zeros([x_extent, y_extent, z_extent, 3])
transmissivity = np.ones([x_extent, y_extent, z_extent, 2])
layer_storage = np.ones([x_extent, y_extent, z_extent, 2])

year_counter = 1
week_counter = 0
annual_tsteps = 1
week_tsteps = annual_tsteps/52
timestep = week_tsteps
t_step_sum = 0
t_step_sum_su = 0
max_tstep = max_tstep/525600
mean_t_step = 0
sub_year_counter = 0
sub_year_counter_su = 0
output_counter = 0
oxic_decay = 0
anoxic_decay = 0
x = 1
y = 1
z = 1
no_layers = 3
new_week = 1

with open("040_DigiBog_BB_IN_column_status.txt","r") as file:
    lines = file.readlines()
    column_status = [line.rstrip() for line in lines]
with open("050_DigiBog_BB_IN_baltitude.txt","r") as file:
    lines = file.readlines()
    strings = [line.rstrip() for line in lines]
    baltitude = [float(x) for x in strings]
grab = 0
for x in range(x_extent):
    for y in range(y_extent):
        activation_status[x,y] = column_status[grab]
        base_altitude[x,y] = baltitude[grab]
        grab = grab+1
for x in range(x_extent):
    for y in range(y_extent):
        if activation_status[x,y]=="on":
            layer_attributes[x,y,no_layers-1,0] = pond_depth
            layer_attributes[x,y,no_layers-1,1] = 0
            layer_attributes[x,y,no_layers-1,2] = 1
for x in range(x_extent):
    for y in range(y_extent):
        if activation_status[x,y]=="on":
            layer_attributes[x,y,0,0] = 2
            layer_attributes[x,y,0,1] = 3153.6
            layer_attributes[x,y,0,2] = 0.3
            transmissivity[x,y,0,0] = layer_attributes[x,y,0,0]
            transmissivity[x,y,0,1] = transmissivity[x,y,0,0]*layer_attributes[x,y,0,1]
            wet_proportion[x,y,0] = 1
            water_table[x,y] = transmissivity[x,y,0,0]
for x in range(x_extent):
    for y in range(y_extent):
        if activation_status[x,y]=="on":
            layer_mass[x,y,1,0] = (9.3**2)*0.0001
            layer_mass[x,y,1,1] = layer_mass[x,y,1,0]
            layer_mass[x,y,1,2] = 1
            layer_attributes[x,y,1,0] = layer_mass[x,y,1,0]/density
            layer_attributes[x,y,1,1] = k_param_a*np.exp(k_param_b)
            layer_attributes[x,y,1,2] = porosity
            transmissivity[x,y,1,0] = transmissivity[x,y,0,0]+layer_attributes[x,y,1,0]
            transmissivity[x,y,1,1] = transmissivity[x,y,0,0]+layer_attributes[x,y,1,0]*layer_attributes[x,y,1,1]
            wet_proportion[x,y,1] = 1
            col_mass_per_area[x,y] = layer_mass[x,y,1,0]
            water_table[x,y] = transmissivity[x,y,1,0]
for x in range(1,(x_extent-1)):
    for y in range(y_extent-1):
        if activation_status[x,y]=="diri":
            water_table[x,y] = transmissivity[x,y+1,1,0]

with open("020_DigiBog_BB_IN_net_rain.txt","r") as file:
    lines = file.readlines()
    strings = [line.rstrip() for line in lines]
    net_rain = [float(x) for x in strings]
with open("030_DigiBog_BB_IN_temp.txt","r") as file:
    lines = file.readlines()
    strings = [line.rstrip() for line in lines]
    temp = [float(x) for x in strings]
grab = 0
t_step_sum_output = []
col_t_step_output = []
column_height_output = []
wt_height_output = []
mass_area_output = []
wt_depth_output = []
wt_depth_summer_output = []
layer_mass_output = []
transmiss_output = []
layer_wet_prop_output = []
k_profile_output = []

while True:
    print("model year",year_counter)
    while True:
        if new_week==1:
            rainfall = net_rain[grab]
            temperature = temp[grab]
            grab = grab+1
            oxic_decay = oxic_decay_base*Q10_oxic**((temperature-base_temp)/10)
            anoxic_decay = anoxic_decay_base*Q10_anoxic**((temperature-base_temp)/10)
            t_step_sum_output.append(t_step_sum)
            t_step_sum = 0
            t_step_sum_su = 0
            new_week = 0
        sub_year_counter = sub_year_counter+1
        if ((week_counter>=17) and (week_counter<=31)):
            sub_year_counter_su = sub_year_counter_su+1
        col_mass_per_area = np.zeros([x_extent, y_extent])
        timestep = week_tsteps
        transmax = 0
        for x in range(x_extent):
            for y in range(y_extent):
                if activation_status[x,y]=="on":
                    layer_storage[x,y,0,0] = layer_attributes[x,y,0,0]
                    layer_storage[x,y,0,1] = layer_attributes[x,y,0,0]*layer_attributes[x,y,0,2]
                    for z in range(1,no_layers):
                        layer_storage[x,y,z,0] = layer_storage[x,y,z-1,0]+layer_attributes[x,y,z,0]
                        layer_storage[x,y,z,1] = layer_attributes[x,y,z,0]*layer_attributes[x,y,z,2]
        for x in range(x_extent):
            for y in range(y_extent):
                if activation_status[x,y]=="on":
                    if water_table[x,y] >= transmissivity[x,y,no_layers-2,0]:
                        wk_mean[x,y] = transmissivity[x,y,no_layers-2,1]
                        if wk_mean[x,y] > transmax:
                            transmax = wk_mean[x,y]
                        wk_mean[x,y] = wk_mean[x,y]/transmissivity[x,y,no_layers-2,0]
                    elif water_table[x,y] < transmissivity[x,y,0,0]:
                        wk_mean[x,y] = layer_attributes[x,y,0,1]
                        if transmissivity[x,y,0,1] > transmax:
                            transmax = transmissivity[x,y,0,1]
                    else:
                        for z in range(no_layers-2):
                            if water_table[x,y] == transmissivity[x,y,z,0]:
                                wk_mean[x,y] = transmissivity[x,y,z,1]
                                if wk_mean[x,y] > transmax:
                                    transmax = wk_mean[x,y]
                                wk_mean[x,y] = wk_mean[x,y]/transmissivity[x,y,z,0]
                            elif water_table[x,y] > transmissivity[x,y,z,0]:
                                if water_table[x,y] < transmissivity[x,y,z+1,0]:
                                    wk_mean[x,y] = transmissivity[x,y,z,1]+layer_attributes[x,y,z+1,1]*(water_table[x,y]-transmissivity[x,y,z,0])
                                    if wk_mean[x,y] > transmax:
                                        transmax = wk_mean[x,y]
                                    wk_mean[x,y] = wk_mean[x,y]/water_table[x,y]
        timestep = 0.5*(0.5*spatial_step)**2*porosity/transmax*0.8
        timestep = timestep*t_step_multi
        if year_counter > 50:
            if timestep > 50/525600:
                timestep = 50/525600
        elif year_counter <= 50:
            if timestep > max_tstep:
                timestep = max_tstep
        if (timestep+t_step_sum) > week_tsteps:
            timestep = week_tsteps-t_step_sum
            new_week = 1
            week_counter = week_counter+1
        t_step_sum = t_step_sum+timestep
        if ((week_counter>=17) and (week_counter<=31)):
            t_step_sum_su = t_step_sum_su+timestep
        x_flux = np.zeros([x_extent, y_extent])
        y_flux = np.zeros([x_extent, y_extent])
        for x in range(x_extent-1):
            for y in range(1,(y_extent-1)):
                if ((activation_status[x,y]=="neu") and (activation_status[x+1,y]=="on")):
                    x_flux[x,y] = 0
                elif ((activation_status[x,y]=="diri") and (activation_status[x+1,y]=="on")):
                    x_flux[x,y] = wk_mean[x+1,y]*((water_table[x,y]+water_table[x+1,y])/2)*(base_altitude[x,y]+water_table[x,y]-base_altitude[x+1,y]-water_table[x+1,y])*2*timestep
                elif ((activation_status[x,y]=="on") and (activation_status[x+1,y]=="neu")):
                    x_flux[x,y] = 0
                elif ((activation_status[x,y]=="on") and (activation_status[x+1,y]=="diri")):
                    x_flux[x,y] = wk_mean[x,y]*((water_table[x,y]+water_table[x+1,y])/2)*(base_altitude[x,y]+water_table[x,y]-base_altitude[x+1,y]-water_table[x+1,y])*2*timestep
                elif ((activation_status[x,y]=="on")&(activation_status[x+1,y]=="on")):
                    x_flux[x,y] = (2*wk_mean[x,y]*wk_mean[x+1,y])/(wk_mean[x,y]+wk_mean[x+1,y])*((water_table[x,y]+water_table[x+1,y])/2)*(base_altitude[x,y]+water_table[x,y]-base_altitude[x+1,y]-water_table[x+1,y])*timestep
        for y in range(y_extent-1):
            for x in range(1,(x_extent-1)):
                if ((activation_status[x,y]=="neu")&(activation_status[x,y+1]=="on")):
                    y_flux[x,y] = 0
                elif ((activation_status[x,y]=="diri")&(activation_status[x,y+1]=="on")):
                    y_flux[x,y] = wk_mean[x,y+1]*((water_table[x,y]+water_table[x,y+1])/2)*(base_altitude[x,y]+water_table[x,y]-base_altitude[x,y+1]-water_table[x,y+1])*2*timestep
                elif ((activation_status[x,y]=="on")&(activation_status[x,y+1]=="neu")):
                    y_flux[x,y] = 0
                elif ((activation_status[x,y]=="on")&(activation_status[x,y+1]=="diri")):
                    y_flux[x,y] = wk_mean[x,y]*((water_table[x,y]+water_table[x,y+1])/2)*(base_altitude[x,y]+water_table[x,y]-base_altitude[x,y+1]-water_table[x,y+1])*2*timestep
                elif ((activation_status[x,y]=="on")&(activation_status[x,y+1]=="on")):
                    y_flux[x,y] = (2*wk_mean[x,y]*wk_mean[x,y+1])/(wk_mean[x,y]+wk_mean[x,y+1])*((water_table[x,y]+water_table[x,y+1])/2)*(base_altitude[x,y]+water_table[x,y]-base_altitude[x,y+1]-water_table[x,y+1])*timestep
        for x in range(1, x_extent-1):
            for y in range(1,(y_extent-1)):
                if activation_status[x,y]=="on":
                    water_change[x,y] = (x_flux[x-1,y]-x_flux[x,y]+y_flux[x,y-1]-y_flux[x,y])/(spatial_step**2)
                    water_change[x,y] = water_change[x,y]+rainfall*timestep
        for x in range(1,(x_extent-1)):
            for y in range(1,(y_extent-1)):
                if activation_status[x,y]=="on":
                    if water_table[x,y] <= layer_storage[x,y,0,0]:
                        marker = 0
                    else:
                        for z in range(no_layers-1):
                            if water_table[x,y] > layer_storage[x,y,z,0]:
                                if water_table[x,y] <= layer_storage[x,y,z+1,0]:
                                    marker = z+1
                                elif (water_table[x,y] > layer_storage[x,y,no_layers-1,0]):
                                    marker = no_layers-1
                                    break
                    if water_change[x,y]>0:
                        if water_table[x,y] < layer_storage[x,y,no_layers-1,0]:
                            z = marker
                            if water_table[x,y]==layer_storage[x,y,z,0]:
                                z=z+1
                            while True:
                                test_depth = layer_storage[x,y,z,1]*(layer_storage[x,y,z,0]-water_table[x,y])/layer_attributes[x,y,z,0]
                                if (water_change[x,y] <= test_depth):
                                    water_table[x,y] = water_table[x,y]+water_change[x,y]/layer_attributes[x,y,z,2]
                                    break
                                else:
                                    if z==(no_layers-1):
                                        water_table[x,y] = layer_storage[x,y,z,0]
                                        break
                                    else:
                                        water_table[x,y] = layer_storage[x,y,z,0]
                                        water_change[x,y] = water_change[x,y]-test_depth
                                        z = z+1
                    if water_change[x,y]<0:
                        if water_table[x,y]>0:
                            z = marker
                            while True:
                                if z==0:
                                    test_depth = layer_storage[x,y,0,1]*water_table[x,y]/layer_attributes[x,y,0,0]
                                else:
                                    test_depth = layer_storage[x,y,z,1]*(water_table[x,y]-layer_storage[x,y,z-1,0])/layer_attributes[x,y,z,0]
                                if abs(water_change[x,y]) <= test_depth:
                                    water_table[x,y] = water_table[x,y]+water_change[x,y]/layer_attributes[x,y,z,2]
                                    break
                                else:
                                    if (z==0):
                                        water_table[x,y] = 0
                                        break
                                    else:
                                        water_table[x,y] = layer_storage[x,y,z-1,0]
                                        water_change[x,y] = water_change[x,y]+test_depth
                                        z = z-1
        for x in range(x_extent):
            for y in range(y_extent):
                if activation_status[x,y]=="on":
                    col_wt_depth[x,y] = transmissivity[x,y,no_layers-2,0]-water_table[x,y]
                    col_wt_sum[x,y] = col_wt_sum[x,y]+col_wt_depth[x,y]
                if ((week_counter >= 17) and (week_counter <= 31)):
                    col_wt_sum_su[x,y] = col_wt_sum_su[x,y]+col_wt_depth[x,y]
        for x in range(x_extent):
            for y in range(y_extent):
                if activation_status[x,y]=="on":
                    for z in range(1,(no_layers-1)):
                        layer_mass[x,y,z,0] = layer_mass[x,y,z,0]*wet_proportion[x,y,z]*np.exp(-anoxic_decay*timestep*layer_mass[x,y,z,2])+layer_mass[x,y,z,0]*(1-wet_proportion[x,y,z])*np.exp(-oxic_decay*timestep)
                        layer_mass[x,y,z,2] = layer_mass[x,y,z,0]/layer_mass[x,y,z,1]
                        layer_attributes[x,y,z,0] = layer_mass[x,y,z,0]/density
                        col_mass_per_area[x,y] = col_mass_per_area[x,y]+layer_mass[x,y,z,0]
        for x in range(x_extent):
            for y in range(y_extent):
                if activation_status[x,y]=="on":
                    for z in range(1,(no_layers-1)):
                        layer_attributes[x,y,z,1] = k_param_a*np.exp(k_param_b*layer_mass[x,y,z,2])
        for x in range(x_extent):
            for y in range(y_extent):
                if activation_status[x,y]=="on":
                    transmissivity[x,y,0,0] = layer_attributes[x,y,0,0]
                    transmissivity[x,y,0,1] = layer_attributes[x,y,0,0]*layer_attributes[x,y,0,1]
                    for z in range(1,(no_layers-1)):
                        transmissivity[x,y,z,0] = transmissivity[x,y,z-1,0]+layer_attributes[x,y,z,0]
                        transmissivity[x,y,z,1] = transmissivity[x,y,z-1,1]+layer_attributes[x,y,z,0]*layer_attributes[x,y,z,1]
        for x in range(x_extent):
            for y in range(y_extent):
                if activation_status[x,y]=="on":
                    for z in range(no_layers-1):
                        if (z==0):
                            if water_table[x,y] >= transmissivity[x,y,z,0]:
                                wet_proportion[x,y,0] = 1
                            else:
                                wet_proportion[x,y,0] = water_table[x,y]/layer_attributes[x,y,0,0]
                        elif water_table[x,y] >= transmissivity[x,y,z,0]:
                            wet_proportion[x,y,z] = 1
                        elif water_table[x,y] <= transmissivity[x,y,z-1,0]:
                            wet_proportion[x,y,z] = 0
                        else:
                            wet_proportion[x,y,z] = (water_table[x,y]-transmissivity[x,y,z-1,0])/layer_attributes[x,y,z,0]
        if week_counter==52:
            break
    for x in range(x_extent):
        for y in range(y_extent):
            if activation_status[x,y]=="on":
                col_wt_depth_av[x,y] = col_wt_sum[x,y]/sub_year_counter
                col_wt_depth_av_su[x,y] = col_wt_sum_su[x,y]/sub_year_counter_su
    mean_t_step = 1/sub_year_counter*525600
    col_t_step_output.append(mean_t_step)
    mean_t_step = 0
    week_counter = 0
    output_counter = output_counter+1
    if output_counter==output_interval:
        for x in range(x_extent):
            for y in range(y_extent):
                if activation_status[x,y]=="on":
                    column_height_output.append(transmissivity[x,y,no_layers-2,0])
                    wt_height_output.append(water_table[x,y])
                    mass_area_output.append(col_mass_per_area[x,y])
                    wt_depth_output.append(col_wt_depth_av[x,y])
                    wt_depth_summer_output.append(col_wt_depth_av_su[x,y])
                else:
                    column_height_output.append(-999)
                    wt_height_output.append(-999)
                    mass_area_output.append(-999)
                    wt_depth_output.append(-999)
                    wt_depth_summer_output.append(-999)
    year_counter = year_counter + 1
    if year_counter > t_extent:
        break
    col_wt_sum = np.zeros([x_extent, y_extent])
    col_wt_sum_su = np.zeros([x_extent, y_extent])
    sub_year_counter = 0
    sub_year_counter_su = 0
    no_layers = no_layers+1
    for x in range(x_extent):
        for y in range(y_extent):
            if activation_status[x,y]=="on":
                layer_attributes[x,y,no_layers-1,0] = pond_depth
                layer_attributes[x,y,no_layers-1,1] = 0
                layer_attributes[x,y,no_layers-1,2] = 1
    for x in range(x_extent):
        for y in range(y_extent):
            if activation_status[x,y]=="on":
                if col_wt_depth_av[x,y] > 66.8:
                    layer_mass[x,y,no_layers-2,0] = 0.0000001
                else:
                    layer_mass[x,y,no_layers-2,0] = (9.3+1.33*col_wt_depth_av[x,y]-0.022*col_wt_depth_av[x,y]**2)**2*0.0001*(0.1575*mean_temp+0.009132)
                layer_mass[x,y,no_layers-2,1] = layer_mass[x,y,no_layers-2,0]
                layer_mass[x,y,no_layers-2,2] = 1
                layer_attributes[x,y,no_layers-2,0] = layer_mass[x,y,no_layers-2,0]/density
                layer_attributes[x,y,no_layers-2,2] = porosity
                transmissivity[x,y,no_layers-2,0] = transmissivity[x,y,no_layers-3,0]+layer_attributes[x,y,no_layers-2,0]
                if water_table[x,y] >= transmissivity[x,y,no_layers-2,0]:
                    wet_proportion[x,y,no_layers-2] = 1
                elif water_table[x,y] <= transmissivity[x,y,no_layers-2,0]:
                    wet_proportion[x,y,z] = 0
                else:
                    wet_proportion[x,y,no_layers-2] = (water_table[x,y]-transmissivity[x,y,no_layers-2,0])/layer_attributes[x,y,no_layers-2,0]
                if wet_proportion[x,y,no_layers-2] > 0:
                    layer_attributes[x,y,no_layers-2,1] = k_param_a*np.exp(k_param_b)
                    transmissivity[x,y,no_layers-2,1] = transmissivity[x,y,no_layers-3,1]+layer_attributes[x,y,no_layers-2,0]*layer_attributes[x,y,no_layers-2,1]
                col_mass_per_area[x,y] = col_mass_per_area[x,y]+layer_mass[x,y,no_layers-2,0]
    output_counter = 0
for x in range(x_extent):
    for y in range(y_extent):
        if activation_status[x,y]=="on":
            for z in range(1,(no_layers-1)):
                layer_mass_output.append(layer_mass[x,y,z,2])
                transmiss_output.append(transmissivity[x,y,z,1])
                layer_wet_prop_output.append(wet_proportion[x,y,z])
                k_profile_output.append(layer_attributes[x,y,z,1])